In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from scipy.stats import norm
import os
import logging
import warnings
from datetime import datetime, timedelta
import pickle
from tqdm import tqdm

# Crear directorios para resultados
os.makedirs('./artifacts/results', exist_ok=True)
os.makedirs('./artifacts/results/figures', exist_ok=True)
os.makedirs('./artifacts/results/data', exist_ok=True)

# Configurar logging
logging.basicConfig(
    filename='./artifacts/errors.txt',
    level=logging.ERROR,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Suprimir advertencias
warnings.filterwarnings('ignore')

class AdaptiveMultifactorStrategy:
    def __init__(self, start_date='2010-01-01', end_date=None, symbols=None, 
                 lookback_window=252, regime_window=63, n_regimes=5, 
                 rebalance_freq=21, vol_target=0.10, max_leverage=1.5,
                 transaction_cost=0.0005, market_impact=0.1, borrow_cost=0.0002,
                 execution_delay=1, use_point_in_time=True, 
                 max_drawdown_limit=0.15, dynamic_vol_scaling=True,
                 risk_targeting=True, regime_detection_method='bgmm'):
        """
        Inicializa la estrategia de descomposición multifactorial adaptativa.
        
        Parámetros:
        -----------
        start_date : str
            Fecha de inicio para los datos históricos (formato 'YYYY-MM-DD')
        end_date : str
            Fecha de fin para los datos históricos (formato 'YYYY-MM-DD')
        symbols : list
            Lista de símbolos a incluir. Si es None, se usa un universo de referencia histórico
        lookback_window : int
            Ventana de observación para calcular factores latentes (días)
        regime_window : int
            Ventana para detectar regímenes de mercado (días)
        n_regimes : int
            Número de regímenes de mercado a detectar
        rebalance_freq : int
            Frecuencia de rebalanceo en días
        vol_target : float
            Volatilidad objetivo anualizada
        max_leverage : float
            Apalancamiento máximo permitido
        transaction_cost : float
            Costo de transacción como porcentaje del valor operado
        market_impact : float
            Impacto de mercado como factor de volatilidad diaria
        borrow_cost : float
            Costo anualizado de tomar posiciones cortas
        execution_delay : int
            Retraso en días entre decisión y ejecución
        use_point_in_time : bool
            Usar datos point-in-time para evitar sesgo de supervivencia
        max_drawdown_limit : float
            Límite de drawdown máximo permitido antes de reducir exposición
        dynamic_vol_scaling : bool
            Usar scaling dinámico de volatilidad para ajustar exposición
        risk_targeting : bool
            Usar targeting dinámico de riesgo basado en regímenes
        regime_detection_method : str
            Método para detectar regímenes ('gmm' o 'bgmm')
        """
        self.start_date = start_date
        self.end_date = end_date if end_date else datetime.now().strftime('%Y-%m-%d')
        self.symbols = symbols
        self.lookback_window = lookback_window
        self.regime_window = regime_window
        self.n_regimes = n_regimes
        self.rebalance_freq = rebalance_freq
        self.vol_target = vol_target
        self.max_leverage = max_leverage
        
        # Parámetros para implementación realista
        self.transaction_cost = transaction_cost
        self.market_impact = market_impact
        self.borrow_cost = borrow_cost
        self.execution_delay = execution_delay
        self.use_point_in_time = use_point_in_time
        
        # Nuevos parámetros de mejora
        self.max_drawdown_limit = max_drawdown_limit
        self.dynamic_vol_scaling = dynamic_vol_scaling
        self.risk_targeting = risk_targeting
        self.regime_detection_method = regime_detection_method
        
        # Atributos que se inicializarán más tarde
        self.prices = None
        self.returns = None
        self.factor_loadings = None
        self.factor_returns = None
        self.regimes = None
        self.regime_probs = None
        self.optimal_weights = None
        self.performance = None
        
        # Parámetros adicionales para simulación realista
        self.tradable_assets = None
        self.max_position_size = 0.1
        self.liquidity_threshold = 1000000
        
        # Cargar datos
        self._load_data()
        
    def _load_data(self):
        """
        Carga los datos históricos de precios y calcula retornos.
        """
        try:
            if self.symbols is None:
                # Obtener S&P 500 actual
                sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
                self.symbols = sp500['Symbol'].tolist()
                
                # Simular cambios en el universo a lo largo del tiempo
                np.random.seed(42)
                # self.symbols = [s for s in self.symbols if np.random.random() > 0.2]
            
            # Descargar datos
            data = yf.download(self.symbols, start=self.start_date, end=self.end_date)
            self.prices = data['Close']
            self.volumes = data['Volume']
            
            # Limpiar y preparar datos
            self.prices = self.prices.dropna(axis=1, thresh=int(len(self.prices) * 0.9))
            self.symbols = list(self.prices.columns)
            
            # Calcular retornos diarios
            self.returns = self.prices.pct_change().dropna()
            
            # Calcular volatilidades diarias para estimar impacto de mercado
            self.daily_vol = self.returns.rolling(21).std().fillna(method='bfill')
            
            # Crear DataFrame para simular disponibilidad de activos
            self.tradable_universe = pd.DataFrame(
                True, 
                index=self.returns.index, 
                columns=self.returns.columns
            )
            
            # Simular disponibilidad para posiciones cortas
            self.shortable_universe = pd.DataFrame(
                # np.random.random(self.tradable_universe.shape) > 0.3,
                True, 
                index=self.tradable_universe.index,
                columns=self.tradable_universe.columns
            )
            
            # Filtrar por liquidez mínima
            if 'Volume' in data.columns:
                dollar_volumes = data['Volume'] * self.prices
                
                # Marca como no negociables los activos con baja liquidez
                for date in self.tradable_universe.index:
                    low_liquidity = dollar_volumes.loc[date] < self.liquidity_threshold
                    self.tradable_universe.loc[date, low_liquidity] = False
                    self.shortable_universe.loc[date, low_liquidity] = False
            
            print(f"Datos cargados exitosamente. {len(self.symbols)} símbolos, {len(self.returns)} días de trading.")
            print(f"En promedio, {self.tradable_universe.mean().mean()*100:.1f}% de los activos son negociables.")
            print(f"En promedio, {self.shortable_universe.mean().mean()*100:.1f}% de los activos son susceptibles de posiciones cortas.")
            
        except Exception as e:
            logging.error(f"Error al cargar datos: {str(e)}", exc_info=True)
            raise
    
    def extract_latent_factors(self, returns_window, n_components=None):
        """
        Extrae factores latentes de los retornos usando PCA.
        
        Parámetros:
        -----------
        returns_window : DataFrame
            Ventana de retornos para extraer factores
        n_components : int, opcional
            Número de componentes a extraer. Si es None, se determina automáticamente.
            
        Retorna:
        --------
        factor_loadings : ndarray
            Cargas de los factores latentes
        factor_returns : DataFrame
            Retornos de los factores latentes
        n_components : int
            Número de componentes utilizados
        """
        try:
            # Manejar valores faltantes
            returns_filled = returns_window.copy()
            
            # Usar imputación por media móvil para NaNs
            for col in returns_filled.columns:
                mask = returns_filled[col].isna()
                if mask.any():
                    returns_filled.loc[mask, col] = returns_filled[col].rolling(5, min_periods=1).mean()[mask]
            
            # Si aún hay NaNs, rellenar con ceros
            returns_filled = returns_filled.fillna(0)
            
            # Determinar número óptimo de componentes si no se especifica
            if n_components is None:
                n_components = self.find_optimal_components(returns_filled)
            
            # Aplicar PCA
            pca = PCA(n_components=n_components)
            factor_returns_np = pca.fit_transform(returns_filled)
            
            # Convertir a DataFrame
            factor_returns = pd.DataFrame(
                factor_returns_np, 
                index=returns_window.index,
                columns=[f'Factor_{i+1}' for i in range(n_components)]
            )
            
            return pca.components_, factor_returns, n_components
            
        except Exception as e:
            logging.error(f"Error en extract_latent_factors: {str(e)}", exc_info=True)
            raise
    
    def find_optimal_components(self, returns_window, threshold=0.85, max_components=15):
        """
        Determina el número óptimo de componentes principales.
        
        Parámetros:
        -----------
        returns_window : DataFrame
            Ventana de retornos para analizar
        threshold : float
            Umbral de varianza explicada acumulada
        max_components : int
            Número máximo de componentes a considerar
            
        Retorna:
        --------
        n_components : int
            Número óptimo de componentes
        """
        try:
            # Limitar el máximo posible de componentes
            max_possible = min(returns_window.shape[1], returns_window.shape[0], max_components)
            
            # Calcular varianza explicada para diferentes números de componentes
            pca = PCA(n_components=max_possible)
            pca.fit(returns_window)
            
            # Encontrar el número de componentes que explican al menos threshold de la varianza
            explained_variance_ratio_cumsum = np.cumsum(pca.explained_variance_ratio_)
            n_components = np.argmax(explained_variance_ratio_cumsum >= threshold) + 1
            
            # Asegurar un mínimo de componentes
            n_components = max(n_components, 3)
            
            return n_components
            
        except Exception as e:
            logging.error(f"Error en find_optimal_components: {str(e)}", exc_info=True)
            # Valor por defecto en caso de error
            return 5
    
    def detect_regimes(self, factor_returns, n_regimes=None):
        """
        Detecta regímenes de mercado usando modelos de mezcla gaussiana.
        
        Parámetros:
        -----------
        factor_returns : DataFrame
            Retornos de los factores latentes
        n_regimes : int, opcional
            Número de regímenes a detectar. Si es None, se usa self.n_regimes.
            
        Retorna:
        --------
        regimes : ndarray
            Etiquetas de régimen para cada punto temporal
        regime_probs : ndarray
            Probabilidades de pertenencia a cada régimen
        """
        try:
            if n_regimes is None:
                n_regimes = self.n_regimes
            
            # MEJORA: Usar más características para detectar regímenes
            # 1. Volatilidad
            vol = factor_returns.rolling(21).std().dropna()
            
            # 2. Correlación entre factores
            # Calcular ventana de correlación
            corr_matrices = []
            for i in range(21, len(factor_returns)):
                window = factor_returns.iloc[i-21:i]
                corr_matrix = window.corr().values
                # Aplanar la parte triangular superior de la matriz
                corr_upper = corr_matrix[np.triu_indices(corr_matrix.shape[0], k=1)]
                corr_matrices.append(corr_upper)
            
            # Convertir a DataFrame
            if corr_matrices:
                corr_df = pd.DataFrame(
                    corr_matrices,
                    index=factor_returns.index[21:],
                    columns=[f'Corr_{i}_{j}' for i, j in zip(*np.triu_indices(factor_returns.shape[1], k=1))]
                )
                
                # Alinear índices
                vol = vol.loc[corr_df.index[0]:].copy()
                
                # Combinar características
                features = pd.concat([vol, corr_df], axis=1)
            else:
                features = vol.copy()
            
            # Estandarizar características
            scaler = StandardScaler()
            features_scaled = scaler.fit_transform(features)
            
            # MEJORA: Usar método más robusto para detectar regímenes
            if self.regime_detection_method == 'bgmm':
                # Bayesian Gaussian Mixture Model es más robusto para detectar regímenes
                gmm = BayesianGaussianMixture(
                    n_components=n_regimes,
                    covariance_type='full',
                    random_state=42,
                    n_init=10,
                    weight_concentration_prior=0.1  # Favorece usar menos componentes si es posible
                )
            else:
                # GMM estándar
                gmm = GaussianMixture(
                    n_components=n_regimes,
                    covariance_type='full',
                    random_state=42,
                    n_init=10
                )
            
            # Manejar NaNs
            features_scaled_clean = np.nan_to_num(features_scaled)
            
            # Ajustar modelo
            gmm.fit(features_scaled_clean)
            
            # Predecir regímenes y probabilidades
            regimes = gmm.predict(features_scaled_clean)
            regime_probs = gmm.predict_proba(features_scaled_clean)
            
            # MEJORA: Ordenar regímenes por volatilidad para que sean consistentes entre ejecuciones
            # Calcular volatilidad media por régimen
            regime_vol = {}
            for r in range(n_regimes):
                regime_mask = (regimes == r)
                if np.any(regime_mask):
                    regime_vol[r] = np.mean(vol.iloc[regime_mask].mean())
                else:
                    regime_vol[r] = 0
            
            # Ordenar regímenes por volatilidad (ascendente)
            regime_map = {old: new for new, (old, _) in enumerate(sorted(regime_vol.items(), key=lambda x: x[1]))}
            
            # Reordenar regímenes y probabilidades
            new_regimes = np.array([regime_map[r] for r in regimes])
            new_probs = np.zeros_like(regime_probs)
            for old, new in regime_map.items():
                new_probs[:, new] = regime_probs[:, old]
            
            return new_regimes, new_probs
            
        except Exception as e:
            logging.error(f"Error en detect_regimes: {str(e)}", exc_info=True)
            # Valores por defecto en caso de error
            dummy_regimes = np.zeros(len(features) if 'features' in locals() else len(factor_returns) - 20)
            dummy_probs = np.ones((len(dummy_regimes), self.n_regimes)) / self.n_regimes
            return dummy_regimes, dummy_probs
    
    def predict_expected_returns(self, returns_window, regimes, current_regime_probs, horizon=10):
        """
        MEJORA: Método de predicción de retornos más robusto con múltiples horizontes
        
        Parámetros:
        -----------
        returns_window : DataFrame
            Ventana histórica de retornos
        regimes : ndarray
            Etiquetas de régimen para cada punto temporal
        current_regime_probs : ndarray
            Probabilidades de pertenencia a cada régimen en el momento actual
        horizon : int
            Horizonte de predicción en días
            
        Retorna:
        --------
        expected_returns : Series
            Retornos esperados para cada activo
        prediction_confidence : Series
            Confianza en las predicciones
        """
        try:
            n_assets = returns_window.shape[1]
            
            # MEJORA: Usar múltiples horizontes para predicción más robusta
            horizons = [5, 10, 21]  # 1 semana, 2 semanas, 1 mes
            horizon_weights = [0.5, 0.3, 0.2]  # Priorizar horizontes más cortos
            
            # Inicializar arrays para retornos esperados por régimen y horizonte
            regime_expected_returns = np.zeros((self.n_regimes, len(horizons), n_assets))
            regime_counts = np.zeros(self.n_regimes)
            
            # Para cada régimen, calcular retornos esperados basados en datos históricos
            for r in range(self.n_regimes):
                # Encontrar índices donde el régimen es r
                regime_indices = np.where(regimes == r)[0]
                regime_counts[r] = len(regime_indices)
                
                if len(regime_indices) > 0:
                    # Para cada horizonte, calcular retornos futuros
                    for h_idx, h in enumerate(horizons):
                        all_future_returns = []
                        
                        for idx in regime_indices:
                            # Solo considerar datos completos para evitar look-ahead bias
                            if idx + h < len(returns_window) - 1:
                                # Próximos 'h' días después de este régimen
                                future_returns = returns_window.iloc[idx+1:idx+1+h].values
                                # Calcular retorno acumulado
                                cum_returns = np.prod(1 + future_returns, axis=0) - 1
                                all_future_returns.append(cum_returns)
                        
                        if all_future_returns:
                            # Promediar los retornos futuros para este régimen y horizonte
                            all_future_returns = np.array(all_future_returns)
                            # MEJORA: Usar mediana para mayor robustez ante outliers
                            regime_expected_returns[r, h_idx] = np.median(all_future_returns, axis=0)
            
            # MEJORA: Calcular retornos esperados ponderados por régimen y horizonte
            expected_returns = np.zeros(n_assets)
            for r in range(self.n_regimes):
                regime_weight = current_regime_probs[r]
                # Ajustar peso por la cantidad de datos (más datos = más confianza)
                confidence_weight = min(1.0, regime_counts[r] / 30)
                
                # Calcular retorno esperado ponderado para cada horizonte
                for h_idx, hw in enumerate(horizon_weights):
                    expected_returns += regime_weight * regime_expected_returns[r, h_idx] * confidence_weight * hw
            
            # MEJORA: Ajustar por autocorrelación - añadir componente de momentum/reversal
            short_momentum = returns_window.iloc[-5:].mean() * 0.2  # Momentum de corto plazo (5 días)
            long_momentum = returns_window.iloc[-21:].mean() * 0.1  # Momentum de largo plazo (21 días)
            
            # Combinar señales
            expected_returns = pd.Series(expected_returns, index=returns_window.columns) + short_momentum - long_momentum
            
            # Calcular confianza de predicción
            regime_certainty = np.max(current_regime_probs)
            data_sufficiency = np.min([count for count in regime_counts if count > 0]) / 30 if any(regime_counts > 0) else 0
            prediction_confidence = pd.Series(regime_certainty * data_sufficiency, index=returns_window.columns)
            
            # MEJORA: Ajustar retornos esperados por volatilidad reciente
            recent_vol = returns_window.iloc[-21:].std()
            vol_scale = np.clip(self.vol_target / (recent_vol * np.sqrt(252)), 0.5, 2.0)
            expected_returns *= vol_scale
            
            return expected_returns, prediction_confidence
            
        except Exception as e:
            logging.error(f"Error en predict_expected_returns: {str(e)}", exc_info=True)
            # Valores por defecto en caso de error
            dummy_returns = pd.Series(0.0001, index=returns_window.columns)
            dummy_confidence = pd.Series(0.1, index=returns_window.columns)
            return dummy_returns, dummy_confidence
    
    def calculate_risk_budget(self, current_regime, regime_certainty, market_vol):
        """
        MEJORA: Calcula dinámicamente el presupuesto de riesgo según condiciones del mercado
        
        Parámetros:
        -----------
        current_regime : int
            Régimen de mercado actual
        regime_certainty : float
            Certeza sobre el régimen actual
        market_vol : float
            Volatilidad reciente del mercado
        
        Retorna:
        --------
        risk_budget : dict
            Parámetros de riesgo para la optimización
        """
        # Volatilidad normalizada (>1 significa alta volatilidad, <1 significa baja volatilidad)
        vol_normalized = market_vol / self.vol_target
        
        # Factor de escala para la aversión al riesgo (mayor en volatilidad alta)
        risk_aversion_scale = np.clip(vol_normalized, 0.5, 3.0)
        
        # Aversión al riesgo base según régimen
        # Regímenes fueron ordenados por volatilidad, así que mayor número = mayor volatilidad
        base_risk_aversion = 1.0 + current_regime * 0.5
        
        # Ajustar por certeza del régimen (menor certeza = mayor aversión)
        certainty_adjustment = 1.0 + (1.0 - regime_certainty) * 2.0
        
        # Aversión al riesgo final
        final_risk_aversion = base_risk_aversion * certainty_adjustment * risk_aversion_scale
        
        # Ajustar leverage máximo según condiciones
        leverage_scale = 1.0
        
        # En regímenes de alta volatilidad, reducir apalancamiento
        if current_regime >= self.n_regimes - 2:  # Dos regímenes más volátiles
            leverage_scale *= 0.7
        
        # Si volatilidad es muy alta, reducir aún más
        if vol_normalized > 1.5:
            leverage_scale *= 0.7
        
        # Si certeza es baja, reducir apalancamiento
        if regime_certainty < 0.5:
            leverage_scale *= 0.9
        
        # Calcular apalancamiento máximo objetivo (nunca mayor al configurado)
        target_max_leverage = min(self.max_leverage * leverage_scale, self.max_leverage)
        
        # Límites para posiciones cortas según régimen y volatilidad
        if current_regime <= 1 and vol_normalized < 1.2:  # Regímenes de baja volatilidad
            short_limit = -0.3  # Permitir más posiciones cortas
        elif current_regime <= 2 and vol_normalized < 1.5:  # Regímenes moderados
            short_limit = -0.2  # Posiciones cortas moderadas
        elif current_regime <= 3:  # Regímenes de mayor volatilidad
            short_limit = -0.1  # Pocas posiciones cortas
        else:  # Regímenes de alta volatilidad
            short_limit = -0.05  # Muy pocas posiciones cortas
        
        # Tamaño máximo de posición ajustado por régimen
        position_size_scale = np.clip(1.5 - current_regime * 0.2, 0.5, 1.0)
        max_position_size = self.max_position_size * position_size_scale
        
        # Devolver presupuesto de riesgo
        return {
            'risk_aversion': final_risk_aversion,
            'max_leverage': target_max_leverage,
            'short_limit': short_limit,
            'max_position_size': max_position_size
        }
    
    def optimize_portfolio(self, expected_returns, factor_loadings, prediction_confidence, 
                          current_regime, regime_certainty, current_date, 
                          previous_weights=None, current_drawdown=0.0,
                          market_vol=0.15):
        """
        MEJORA: Optimiza el portafolio con control dinámico de riesgo
        
        Parámetros:
        -----------
        expected_returns : Series
            Retornos esperados para cada activo
        factor_loadings : ndarray
            Cargas de los factores latentes
        prediction_confidence : Series
            Confianza en las predicciones
        current_regime : int
            Régimen de mercado actual
        regime_certainty : float
            Certeza sobre el régimen actual
        current_date : Timestamp
            Fecha actual para determinar activos negociables
        previous_weights : Series, opcional
            Pesos del portafolio previo
        current_drawdown : float
            Drawdown actual del portafolio
        market_vol : float
            Volatilidad reciente del mercado anualizada
            
        Retorna:
        --------
        weights : Series
            Pesos óptimos para cada activo
        """
        try:
            n_assets = len(expected_returns)
            assets = expected_returns.index
            
            # Determinar activos negociables en la fecha actual
            if current_date in self.tradable_universe.index:
                tradable_assets = assets[self.tradable_universe.loc[current_date, assets]]
                shortable_assets = assets[self.shortable_universe.loc[current_date, assets]]
            else:
                # Si fecha no disponible, usar más reciente
                last_available = self.tradable_universe.index[self.tradable_universe.index <= current_date]
                if len(last_available) > 0:
                    last_available = last_available[-1]
                    tradable_assets = assets[self.tradable_universe.loc[last_available, assets]]
                    shortable_assets = assets[self.shortable_universe.loc[last_available, assets]]
                else:
                    # Fallback: asumir todos negociables
                    tradable_assets = assets
                    shortable_assets = assets
            
            # Filtrar activos no negociables
            filtered_returns = expected_returns.copy()
            for asset in assets:
                if asset not in tradable_assets:
                    filtered_returns[asset] = np.nan
            
            # Rellenar NaNs con valores muy negativos para evitar seleccionarlos
            filtered_returns = filtered_returns.fillna(-999)
            
            # MEJORA: Usar presupuesto de riesgo dinámico
            risk_budget = self.calculate_risk_budget(current_regime, regime_certainty, market_vol)
            adjusted_risk_aversion = risk_budget['risk_aversion']
            target_max_leverage = risk_budget['max_leverage']
            short_limit = risk_budget['short_limit']
            max_position_size = risk_budget['max_position_size']
            
            # MEJORA: Reducir exposición si drawdown se acerca al límite
            drawdown_factor = 1.0
            if self.max_drawdown_limit > 0 and current_drawdown > 0:
                # Reducir exposición progresivamente a medida que se acerca al límite
                drawdown_ratio = current_drawdown / self.max_drawdown_limit
                if drawdown_ratio > 0.5:
                    drawdown_factor = max(0.5, 1.0 - (drawdown_ratio - 0.5) * 2.0)
                    print(f"Reduciendo exposición por drawdown: factor = {drawdown_factor:.2f}")
            
            # Ajustar apalancamiento máximo por drawdown
            target_max_leverage *= drawdown_factor
            
            # Calcular matriz de covarianza basada en factores latentes
            factor_cov = np.cov(factor_loadings)
            asset_cov = factor_loadings.T @ factor_cov @ factor_loadings
            
            # Asegurar que la matriz es definida positiva
            asset_cov = (asset_cov + asset_cov.T) / 2
            min_eig = np.min(np.real(np.linalg.eigvals(asset_cov)))
            if min_eig < 1e-6:
                asset_cov += np.eye(n_assets) * (1e-6 - min_eig)
            
            # Ajustar retornos esperados por confianza
            adjusted_returns = filtered_returns * prediction_confidence
            
            # Considerar costos de préstamo para posiciones cortas
            borrow_costs = pd.Series(0.0, index=assets)
            for asset in assets:
                if asset not in shortable_assets:
                    borrow_costs[asset] = 0.1  # Penalización alta para activos no shortables
                else:
                    borrow_costs[asset] = self.borrow_cost / 252
            
            # Incluir costos de transacción estimados
            if previous_weights is not None:
                # Estimar impacto de mercado basado en volatilidad
                market_impact_cost = pd.Series(0.0, index=assets)
                for asset in assets:
                    if asset in self.daily_vol.columns and current_date in self.daily_vol.index:
                        vol = self.daily_vol.loc[current_date, asset]
                        market_impact_cost[asset] = vol * self.market_impact
                    else:
                        market_impact_cost[asset] = 0.02 * self.market_impact
            else:
                market_impact_cost = pd.Series(0.0, index=assets)
                previous_weights = pd.Series(0.0, index=assets)
            
            # MEJORA: Función objetivo mejorada con más controles
            def objective(weights):
                weights_series = pd.Series(weights, index=assets)
                
                # Retorno esperado
                portfolio_return = np.sum(weights_series * adjusted_returns)
                
                # Riesgo
                portfolio_risk = np.sqrt(weights_series.T @ asset_cov @ weights_series)
                
                # Costos de transacción
                turnover = np.sum(np.abs(weights_series - previous_weights))
                transaction_costs = turnover * self.transaction_cost
                
                # Impacto de mercado estimado
                impact_costs = np.sum(np.abs(weights_series - previous_weights) * market_impact_cost)
                
                # Costos de préstamo para posiciones cortas
                short_costs = np.sum(np.maximum(-weights_series, 0) * borrow_costs)
                
                # MEJORA: Penalización para diversificación
                concentration_penalty = np.sum(weights_series ** 2) * adjusted_risk_aversion * 0.5
                
                # Utilidad final: retorno - riesgo - costos - concentración
                utility = (portfolio_return 
                           - adjusted_risk_aversion * portfolio_risk 
                           - transaction_costs 
                           - impact_costs 
                           - short_costs
                           - concentration_penalty)
                
                return -utility  # Negativo porque minimizamos
            
            # Restricciones
            constraints = [
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}  # Suma de pesos = 1
            ]
            
            # Límites en las posiciones
            bounds = []
            for asset in assets:
                if asset not in tradable_assets:
                    # Activo no negociable
                    bounds.append((0, 0))
                elif asset not in shortable_assets:
                    # Activo negociable pero no shortable
                    bounds.append((0, min(1.0, max_position_size)))
                else:
                    # Activo completamente negociable
                    # Ajustar límites según régimen actual
                    asset_short_limit = short_limit
                    bounds.append((max(asset_short_limit, -max_position_size), min(1.0, max_position_size)))
            
            # Solución inicial: pesos previos o iguales si no hay previos
            if previous_weights is not None and not previous_weights.isna().any():
                initial_weights = previous_weights.values
            else:
                # Sólo asignar a activos negociables
                initial_weights = np.zeros(n_assets)
                tradable_indices = [i for i, asset in enumerate(assets) if asset in tradable_assets]
                if tradable_indices:
                    initial_weights[tradable_indices] = 1.0 / len(tradable_indices)
            
            # Optimizar
            result = minimize(
                objective,
                initial_weights,
                method='SLSQP',
                bounds=bounds,
                constraints=constraints,
                options={'maxiter': 1000, 'ftol': 1e-8}
            )
            
            if not result.success:
                logging.warning(f"Optimización no convergió: {result.message}")
                # Usar pesos iniciales como fallback
                optimal_weights = pd.Series(initial_weights, index=assets)
            else:
                optimal_weights = pd.Series(result.x, index=assets)
            
            # Eliminar posiciones muy pequeñas (menor a 0.1%)
            small_positions = np.abs(optimal_weights) < 0.001
            optimal_weights[small_positions] = 0.0
            
            # Renormalizar para asegurar suma = 1.0
            if optimal_weights.sum() != 0:
                optimal_weights = optimal_weights / optimal_weights.sum()
            
            # MEJORA: Aplicar control de volatilidad objetivo
            expected_vol = np.sqrt(optimal_weights.T @ asset_cov @ optimal_weights) * np.sqrt(252)
            
            if expected_vol > 0:
                # Escalar según volatilidad objetivo
                vol_scalar = min(self.vol_target / expected_vol, target_max_leverage)
                
                # MEJORA: Reducir volatilidad en regímenes de alta volatilidad
                if current_regime >= self.n_regimes - 1:  # Régimen más volátil
                    vol_scalar *= 0.7
                elif current_regime >= self.n_regimes - 2:  # Segundo régimen más volátil
                    vol_scalar *= 0.85
            else:
                vol_scalar = 1.0
            
            # MEJORA: Asegurar que el apalancamiento final nunca excede el máximo permitido
            # Esto es clave para arreglar el problema principal
            leverage = min(vol_scalar, target_max_leverage)
            
            # Calcular apalancamiento bruto (suma de valores absolutos)
            gross_leverage = np.sum(np.abs(optimal_weights * leverage))
            
            # Verificar que no excede el máximo permitido
            if gross_leverage > self.max_leverage:
                # Reducir proporcionalmente
                correction_factor = self.max_leverage / gross_leverage
                leverage *= correction_factor
                print(f"AVISO: Apalancamiento corregido de {gross_leverage:.2f} a {self.max_leverage:.2f}")
            
            # Ajustar pesos finales
            final_weights = optimal_weights * leverage
            
            # Verificación final de apalancamiento bruto
            final_gross_leverage = np.sum(np.abs(final_weights))
            if final_gross_leverage > self.max_leverage + 1e-6:
                logging.warning(f"Apalancamiento final {final_gross_leverage:.4f} excede el máximo {self.max_leverage}")
                # Corrección de emergencia
                final_weights = final_weights * (self.max_leverage / final_gross_leverage)
            
            return final_weights
            
        except Exception as e:
            logging.error(f"Error en optimize_portfolio: {str(e)}", exc_info=True)
            # Valor por defecto: pesos iguales en activos negociables
            if current_date in self.tradable_universe.index:
                tradable_mask = self.tradable_universe.loc[current_date, assets]
            else:
                tradable_mask = pd.Series(True, index=assets)
            
            default_weights = pd.Series(0.0, index=assets)
            tradable_assets = assets[tradable_mask]
            if len(tradable_assets) > 0:
                default_weights[tradable_assets] = 1.0 / len(tradable_assets)
            return default_weights
    
    def check_regime_change_rebalance(self, current_date_idx, backtest_dates, current_regime, regimes_history):
        """
        MEJORA: Verifica si se debe rebalancear por cambio de régimen.
        
        Parámetros:
        -----------
        current_date_idx : int
            Índice de la fecha actual
        backtest_dates : DatetimeIndex
            Fechas del backtest
        current_regime : int
            Régimen actual
        regimes_history : list
            Historial de regímenes
            
        Retorna:
        --------
        should_rebalance : bool
            True si se debe rebalancear por cambio de régimen
        """
        # Si es la primera fecha, no hay cambio de régimen
        if current_date_idx == 0 or len(regimes_history) == 0:
            return False
        
        # Si ya pasaron menos de 5 días desde el último rebalanceo, no rebalancear
        last_rebalance_idx = max([i for i, r in enumerate(regimes_history) if r is not None], default=-1)
        if current_date_idx - last_rebalance_idx < 5:
            return False
        
        # Verificar si el régimen ha cambiado desde el último rebalanceo
        last_regime = next((r for r in reversed(regimes_history) if r is not None), None)
        if last_regime is not None and current_regime != last_regime:
            # Solo rebalancear si el cambio es significativo (distancia > 1)
            if abs(current_regime - last_regime) > 1:
                print(f"Rebalanceo por cambio de régimen: {last_regime} -> {current_regime}")
                return True
        
        return False
    
    def calculate_dynamic_rebalance_freq(self, current_regime, market_vol):
        """
        MEJORA: Calcula frecuencia de rebalanceo dinámica basada en régimen y volatilidad.
        
        Parámetros:
        -----------
        current_regime : int
            Régimen actual
        market_vol : float
            Volatilidad anualizada reciente
            
        Retorna:
        --------
        rebalance_freq : int
            Frecuencia de rebalanceo en días
        """
        base_freq = self.rebalance_freq
        
        # En regímenes de alta volatilidad, rebalancear más frecuentemente
        if current_regime >= self.n_regimes - 1:  # Régimen más volátil
            base_freq = max(5, base_freq // 3)
        elif current_regime >= self.n_regimes - 2:  # Segundo régimen más volátil
            base_freq = max(10, base_freq // 2)
        
        # Si volatilidad es muy alta o muy baja, ajustar frecuencia
        vol_ratio = market_vol / self.vol_target
        if vol_ratio > 2.0:
            base_freq = max(5, base_freq // 2)
        elif vol_ratio < 0.5:
            base_freq = min(42, base_freq * 2)
        
        return base_freq
    
    def backtest(self, start_date=None, end_date=None):
        """
        MEJORA: Backtest con control dinámico de riesgo y rebalanceo adaptativo.
        
        Parámetros:
        -----------
        start_date : str, opcional
            Fecha de inicio del backtest (formato 'YYYY-MM-DD')
        end_date : str, opcional
            Fecha de fin del backtest (formato 'YYYY-MM-DD')
            
        Retorna:
        --------
        performance : DataFrame
            Resultados del backtest incluyendo retornos, drawdowns, etc.
        """
        try:
            # Configurar fechas
            if start_date is None:
                start_date = self.returns.index[self.lookback_window]
            else:
                start_date = pd.to_datetime(start_date)
            
            if end_date is None:
                end_date = self.returns.index[-1]
            else:
                end_date = pd.to_datetime(end_date)
            
            # Filtrar datos por fechas
            mask = (self.returns.index >= start_date) & (self.returns.index <= end_date)
            backtest_dates = self.returns.index[mask]
            
            # Inicializar resultados
            portfolio_values = [1.0]
            portfolio_returns = []
            weights_history = []
            regime_history = []
            drawdown_history = [0.0]
            leverage_history = []
            pending_orders = {}
            
            # Inicializar pesos (comenzar con efectivo)
            current_weights = pd.Series(0, index=self.returns.columns)
            
            # Para rebalanceo dinámico
            next_rebalance_date = None
            dynamic_rebalance_freq = self.rebalance_freq
            
            # Ejecutar backtest
            for i, date in enumerate(tqdm(backtest_dates)):
                # Manejar órdenes pendientes
                if date in pending_orders:
                    order_date, target_weights = pending_orders[date]
                    current_weights = target_weights.copy()
                    del pending_orders[date]
                
                # Determinar si es momento de rebalancear
                should_rebalance = False
                
                # Rebalancear en la primera fecha
                if i == 0:
                    should_rebalance = True
                    next_rebalance_date = None
                
                # MEJORA: Rebalanceo con frecuencia dinámica
                if next_rebalance_date is None or date >= next_rebalance_date:
                    should_rebalance = True
                
                # Obtener datos hasta la fecha actual
                current_idx = self.returns.index.get_loc(date)
                history_end_idx = current_idx
                history_start_idx = max(0, history_end_idx - self.lookback_window)
                
                returns_window = self.returns.iloc[history_start_idx:history_end_idx]
                
                # Calcular volatilidad del mercado para ajustes dinámicos
                market_vol = returns_window.mean(axis=1).rolling(21).std().iloc[-1] * np.sqrt(252) if len(returns_window) > 21 else 0.15
                
                # Detectar regímenes solo si es necesario para rebalanceo dinámico o si vamos a rebalancear
                if should_rebalance or i % 5 == 0:  # Verificar cada 5 días para detección de cambios de régimen
                    # Extraer factores latentes
                    factor_loadings, factor_returns, n_components = self.extract_latent_factors(returns_window)
                    
                    # Detectar regímenes
                    regimes, regime_probs = self.detect_regimes(factor_returns)
                    
                    # Determinar régimen actual
                    current_regime = regimes[-1]
                    regime_certainty = np.max(regime_probs[-1])
                    
                    # MEJORA: Verificar si se debe rebalancear por cambio de régimen
                    if self.check_regime_change_rebalance(i, backtest_dates, current_regime, regime_history):
                        should_rebalance = True
                    
                    # MEJORA: Actualizar frecuencia de rebalanceo dinámico
                    if should_rebalance:
                        dynamic_rebalance_freq = self.calculate_dynamic_rebalance_freq(current_regime, market_vol)
                        next_rebalance_date = backtest_dates[min(i + dynamic_rebalance_freq, len(backtest_dates) - 1)]
                else:
                    # Si no calculamos regímenes, usar el último conocido
                    current_regime = regime_history[-1] if regime_history else 0
                    regime_certainty = 0.8  # Valor por defecto
                
                # Rebalancear si es necesario
                if should_rebalance:
                    # Recalcular factores y regímenes si no lo hicimos antes
                    if 'factor_loadings' not in locals():
                        factor_loadings, factor_returns, n_components = self.extract_latent_factors(returns_window)
                        regimes, regime_probs = self.detect_regimes(factor_returns)
                        current_regime = regimes[-1]
                        regime_certainty = np.max(regime_probs[-1])
                    
                    # Calcular retornos esperados
                    expected_returns, prediction_confidence = self.predict_expected_returns(
                        returns_window, regimes, regime_probs[-1], horizon=10
                    )
                    
                    # MEJORA: Pasar drawdown actual para control de riesgo dinámico
                    current_drawdown = drawdown_history[-1]
                    
                    # Optimizar portafolio
                    target_weights = self.optimize_portfolio(
                        expected_returns,
                        factor_loadings,
                        prediction_confidence,
                        current_regime,
                        regime_certainty,
                        date,
                        current_weights,
                        current_drawdown,
                        market_vol
                    )
                    
                    # Simular retraso en la ejecución
                    if self.execution_delay > 0 and i + self.execution_delay < len(backtest_dates):
                        execution_date = backtest_dates[i + self.execution_delay]
                        pending_orders[execution_date] = (date, target_weights)
                    else:
                        current_weights = target_weights.copy()
                    
                    # Guardar régimen actual
                    regime_history.append(current_regime)
                else:
                    # Si no rebalanceamos, el régimen es None para ese día
                    regime_history.append(None)
                
                # Calcular costos de posiciones cortas
                short_positions = current_weights[current_weights < 0]
                short_cost = 0
                if not short_positions.empty:
                    daily_borrow_cost = self.borrow_cost / 252
                    short_cost = (short_positions.abs() * daily_borrow_cost).sum()
                
                # Guardar apalancamiento actual
                current_leverage = np.sum(np.abs(current_weights))
                leverage_history.append(current_leverage)
                
                # Calcular retorno del portafolio para el día siguiente
                if i + 1 < len(backtest_dates):
                    next_date = backtest_dates[i + 1]
                    next_returns = self.returns.loc[next_date]
                    
                    # Incluir costos de transacción si hubo rebalanceo
                    transaction_cost = 0
                    if should_rebalance:
                        weights_before = weights_history[-1] if weights_history else pd.Series(0, index=current_weights.index)
                        turnover = np.sum(np.abs(current_weights - weights_before))
                        transaction_cost = turnover * self.transaction_cost
                    
                    # Calcular retorno del portafolio con costos
                    portfolio_return = (current_weights * next_returns).sum() - short_cost - transaction_cost
                    portfolio_returns.append(portfolio_return)
                    
                    # Actualizar valor del portafolio
                    portfolio_values.append(portfolio_values[-1] * (1 + portfolio_return))
                    
                    # Calcular drawdown actual
                    peak = max(portfolio_values)
                    current_drawdown = 1 - portfolio_values[-1] / peak
                    drawdown_history.append(current_drawdown)
                    
                    # MEJORA: Reducir exposición si drawdown supera el límite
                    if self.max_drawdown_limit > 0 and current_drawdown > self.max_drawdown_limit:
                        print(f"ALERTA: Drawdown {current_drawdown:.2%} excede límite {self.max_drawdown_limit:.2%}. Reduciendo exposición.")
                        # Reducir exposición en un 30%
                        current_weights = current_weights * 0.7
                        # Forzar rebalanceo en próxima fecha
                        next_rebalance_date = backtest_dates[min(i + 1, len(backtest_dates) - 1)]
                
                # Guardar pesos
                weights_history.append(current_weights.copy())
            
            # Crear DataFrame de resultados
            performance = pd.DataFrame({
                'Portfolio_Value': portfolio_values[:-1],  # Ajustar longitud
                'Returns': portfolio_returns,
                'Leverage': leverage_history[:-1]  # Ajustar longitud
            }, index=backtest_dates[:-1])  # Ajustar fechas
            
            # Calcular métricas
            performance['Cumulative_Returns'] = (1 + performance['Returns']).cumprod()
            performance['Drawdown'] = 1 - performance['Cumulative_Returns'] / performance['Cumulative_Returns'].cummax()
            
            # Guardar resultados adicionales
            self.weights_history = pd.DataFrame(weights_history, index=backtest_dates)
            
            # Convertir regime_history a Series, manejando valores None
            valid_regimes = [(i, r) for i, r in enumerate(regime_history) if r is not None]
            valid_indices = [backtest_dates[i] for i, _ in valid_regimes]
            valid_values = [r for _, r in valid_regimes]
            self.regime_history = pd.Series(valid_values, index=valid_indices)
            
            self.performance = performance
            
            return performance
            
        except Exception as e:
            logging.error(f"Error en backtest: {str(e)}", exc_info=True)
            raise
    
    def calculate_metrics(self, performance=None):
        """
        Calcula métricas de rendimiento de la estrategia.
        
        Parámetros:
        -----------
        performance : DataFrame, opcional
            Resultados del backtest. Si es None, se usa self.performance.
            
        Retorna:
        --------
        metrics : dict
            Diccionario con métricas de rendimiento
        """
        try:
            if performance is None:
                performance = self.performance
            
            if performance is None or len(performance) == 0:
                raise ValueError("No hay datos de rendimiento disponibles")
            
            # Calcular métricas anualizadas
            returns = performance['Returns']
            ann_factor = 252
            
            total_return = performance['Cumulative_Returns'].iloc[-1] - 1
            ann_return = (1 + total_return) ** (ann_factor / len(returns)) - 1
            ann_volatility = returns.std() * np.sqrt(ann_factor)
            sharpe_ratio = ann_return / ann_volatility if ann_volatility > 0 else 0
            max_drawdown = performance['Drawdown'].max()
            
            # Calcular ratio de Sortino (solo considera volatilidad negativa)
            negative_returns = returns[returns < 0]
            downside_deviation = negative_returns.std() * np.sqrt(ann_factor) if len(negative_returns) > 0 else 0
            sortino_ratio = ann_return / downside_deviation if downside_deviation > 0 else 0
            
            # Calcular ratio de Calmar (retorno anualizado / máximo drawdown)
            calmar_ratio = ann_return / max_drawdown if max_drawdown > 0 else 0
            
            # Estimar turnover anualizado (rotación de cartera)
            if hasattr(self, 'weights_history') and len(self.weights_history) > 1:
                turnovers = []
                for i in range(1, len(self.weights_history)):
                    turnover = np.sum(np.abs(self.weights_history.iloc[i] - self.weights_history.iloc[i-1]))
                    turnovers.append(turnover)
                avg_turnover = np.mean(turnovers) if turnovers else 0
                avg_rebalance_freq = self.rebalance_freq  # Usar promedio si tuviéramos frecuencia dinámica
                ann_turnover = avg_turnover * (252 / avg_rebalance_freq)
            else:
                ann_turnover = 0
            
            # Calcular % de meses positivos
            monthly_returns = returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
            pct_positive_months = (monthly_returns > 0).mean() if len(monthly_returns) > 0 else 0
            
            # Calcular métricas realistas
            gross_return = ann_return
            
            # Estimar costos anuales
            estimated_transaction_costs = ann_turnover * self.transaction_cost
            
            # Estimar costos de préstamo para posiciones cortas
            if hasattr(self, 'weights_history'):
                short_exposure = self.weights_history.apply(lambda x: np.sum(np.maximum(-x, 0)), axis=1).mean()
                short_costs = short_exposure * self.borrow_cost
            else:
                short_costs = 0
            
            # Retorno neto
            net_return = gross_return - estimated_transaction_costs - short_costs
            net_sharpe = net_return / ann_volatility if ann_volatility > 0 else 0
            
            # Calcular apalancamiento promedio
            avg_leverage = performance['Leverage'].mean() if 'Leverage' in performance.columns else 0
            max_leverage_used = performance['Leverage'].max() if 'Leverage' in performance.columns else 0
            
            # Calcular características de drawdown
            dd = performance['Drawdown']
            avg_drawdown = dd.mean()
            drawdown_duration = 0
            if max_drawdown > 0:
                # Encontrar períodos de drawdown
                dd_periods = []
                in_dd = False
                start_idx = 0
                for i, val in enumerate(dd):
                    if not in_dd and val > 0.01:  # Inicio de drawdown (>1%)
                        in_dd = True
                        start_idx = i
                    elif in_dd and val < 0.01:  # Fin de drawdown
                        in_dd = False
                        dd_periods.append(i - start_idx)
                
                # Calcular duración promedio en días
                drawdown_duration = np.mean(dd_periods) if dd_periods else 0
            
            # Recopilar métricas
            metrics = {
                'Gross Total Return': total_return,
                'Gross Annualized Return': gross_return,
                'Net Annualized Return': net_return,
                'Annualized Volatility': ann_volatility,
                'Gross Sharpe Ratio': sharpe_ratio,
                'Net Sharpe Ratio': net_sharpe,
                'Sortino Ratio': sortino_ratio,
                'Calmar Ratio': calmar_ratio,
                'Maximum Drawdown': max_drawdown,
                'Average Drawdown': avg_drawdown,
                'Average Drawdown Duration (days)': drawdown_duration,
                'Annualized Turnover': ann_turnover,
                'Estimated Transaction Costs': estimated_transaction_costs,
                'Estimated Short Costs': short_costs,
                'Positive Months (%)': pct_positive_months,
                'Average Leverage': avg_leverage,
                'Maximum Leverage Used': max_leverage_used,
                'Number of Rebalances': len(self.regime_history) if hasattr(self, 'regime_history') else 0
            }
            
            return metrics
            
        except Exception as e:
            logging.error(f"Error en calculate_metrics: {str(e)}", exc_info=True)
            return {}
    
    def plot_results(self, save_path='./artifacts/results/figures/'):
        """
        Genera y guarda visualizaciones de los resultados.
        
        Parámetros:
        -----------
        save_path : str
            Ruta donde guardar las figuras
        """
        try:
            if self.performance is None or len(self.performance) == 0:
                raise ValueError("No hay datos de rendimiento disponibles")
            
            # Crear directorio si no existe
            os.makedirs(save_path, exist_ok=True)
            
            # 1. Gráfico de rendimiento acumulado
            plt.figure(figsize=(12, 6))
            self.performance['Cumulative_Returns'].plot()
            plt.title('Rendimiento Acumulado')
            plt.xlabel('Fecha')
            plt.ylabel('Retorno Acumulado')
            plt.grid(True)
            plt.savefig(f'{save_path}cumulative_returns.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 2. Gráfico de drawdowns
            plt.figure(figsize=(12, 6))
            self.performance['Drawdown'].plot(color='red')
            plt.title('Drawdowns')
            plt.xlabel('Fecha')
            plt.ylabel('Drawdown')
            plt.grid(True)
            plt.savefig(f'{save_path}drawdowns.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 3. Gráfico de regímenes de mercado
            if hasattr(self, 'regime_history') and len(self.regime_history) > 0:
                plt.figure(figsize=(12, 6))
                self.regime_history.plot(drawstyle='steps', marker='o', markersize=4)
                plt.title('Regímenes de Mercado Detectados')
                plt.xlabel('Fecha')
                plt.ylabel('Régimen')
                plt.yticks(range(self.n_regimes))
                plt.grid(True)
                plt.savefig(f'{save_path}market_regimes.png', dpi=300, bbox_inches='tight')
                plt.close()
                
                # 3b. Superponer regímenes con retornos acumulados
                plt.figure(figsize=(14, 8))
                ax1 = plt.gca()
                self.performance['Cumulative_Returns'].plot(ax=ax1, color='blue')
                ax1.set_xlabel('Fecha')
                ax1.set_ylabel('Retorno Acumulado', color='blue')
                ax1.tick_params(axis='y', labelcolor='blue')
                
                ax2 = ax1.twinx()
                # Solo usar fechas que existen en ambos índices
                common_dates = self.regime_history.index.intersection(self.performance.index)
                if len(common_dates) > 0:
                    self.regime_history.loc[common_dates].plot(ax=ax2, color='red', drawstyle='steps', alpha=0.7, marker='o', markersize=3)
                    ax2.set_ylabel('Régimen', color='red')
                    ax2.tick_params(axis='y', labelcolor='red')
                    ax2.set_yticks(range(self.n_regimes))
                
                plt.title('Rendimiento vs. Regímenes de Mercado')
                plt.grid(True)
                plt.savefig(f'{save_path}returns_vs_regimes.png', dpi=300, bbox_inches='tight')
                plt.close()
            
            # 4. Gráfico de exposición a activos a lo largo del tiempo
            if hasattr(self, 'weights_history') and len(self.weights_history) > 0:
                # Seleccionar los 10 activos con mayor peso promedio absoluto
                top_assets = self.weights_history.abs().mean().nlargest(10).index
                
                plt.figure(figsize=(12, 8))
                self.weights_history[top_assets].plot(colormap='viridis')
                plt.title('Exposición a los 10 Activos Principales')
                plt.xlabel('Fecha')
                plt.ylabel('Peso en el Portafolio')
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
                plt.grid(True)
                plt.savefig(f'{save_path}asset_exposure.png', dpi=300, bbox_inches='tight')
                plt.close()
                
                # 5. Heatmap de pesos a lo largo del tiempo
                plt.figure(figsize=(14, 10))
                sns.heatmap(
                    self.weights_history[top_assets].T,
                    cmap='RdBu_r',
                    center=0,
                    robust=True,
                    cbar_kws={'label': 'Peso'}
                )
                plt.title('Evolución de Pesos del Portafolio (Top 10 Activos)')
                plt.xlabel('Tiempo')
                plt.ylabel('Activo')
                plt.savefig(f'{save_path}weights_heatmap.png', dpi=300, bbox_inches='tight')
                plt.close()
                
                # 6. Gráfico de apalancamiento a lo largo del tiempo
                leverage = self.weights_history.abs().sum(axis=1)
                plt.figure(figsize=(12, 6))
                leverage.plot()
                plt.axhline(y=self.max_leverage, color='r', linestyle='--', label=f'Límite máximo ({self.max_leverage})')
                plt.axhline(y=1.0, color='g', linestyle='--', label='Sin apalancamiento')
                plt.title('Apalancamiento del Portafolio')
                plt.xlabel('Fecha')
                plt.ylabel('Apalancamiento')
                plt.legend()
                plt.grid(True)
                plt.savefig(f'{save_path}leverage.png', dpi=300, bbox_inches='tight')
                plt.close()
                
                # 7. Gráfico de exposición neta y bruta
                gross_exposure = self.weights_history.abs().sum(axis=1)
                net_exposure = self.weights_history.sum(axis=1)
                
                plt.figure(figsize=(12, 6))
                gross_exposure.plot(label='Exposición Bruta')
                net_exposure.plot(label='Exposición Neta')
                plt.axhline(y=1.0, color='r', linestyle='--')
                plt.title('Exposición Neta y Bruta del Portafolio')
                plt.xlabel('Fecha')
                plt.ylabel('Exposición')
                plt.legend()
                plt.grid(True)
                plt.savefig(f'{save_path}net_gross_exposure.png', dpi=300, bbox_inches='tight')
                plt.close()
            
            # 8. Distribución de retornos
            plt.figure(figsize=(12, 6))
            sns.histplot(self.performance['Returns'], kde=True)
            plt.title('Distribución de Retornos Diarios')
            plt.xlabel('Retorno')
            plt.ylabel('Frecuencia')
            plt.grid(True)
            plt.savefig(f'{save_path}returns_distribution.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 9. QQ-Plot de retornos vs distribución normal
            plt.figure(figsize=(10, 10))
            from scipy import stats
            stats.probplot(self.performance['Returns'].dropna(), dist="norm", plot=plt)
            plt.title('QQ-Plot de Retornos vs Distribución Normal')
            plt.grid(True)
            plt.savefig(f'{save_path}qqplot.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 10. MEJORA: Gráfico de drawdown vs. apalancamiento
            plt.figure(figsize=(12, 6))
            ax1 = plt.gca()
            self.performance['Drawdown'].plot(ax=ax1, color='red', label='Drawdown')
            ax1.set_xlabel('Fecha')
            ax1.set_ylabel('Drawdown', color='red')
            ax1.tick_params(axis='y', labelcolor='red')
            ax1.legend(loc='upper left')
            
            ax2 = ax1.twinx()
            self.performance['Leverage'].plot(ax=ax2, color='blue', label='Apalancamiento')
            ax2.set_ylabel('Apalancamiento', color='blue')
            ax2.tick_params(axis='y', labelcolor='blue')
            ax2.legend(loc='upper right')
            
            plt.title('Drawdown vs. Apalancamiento')
            plt.grid(True)
            plt.savefig(f'{save_path}drawdown_vs_leverage.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 11. MEJORA: Retornos móviles anualizados
            rolling_returns = self.performance['Returns'].rolling(252).apply(lambda x: (1 + x).prod() - 1)
            rolling_vol = self.performance['Returns'].rolling(252).std() * np.sqrt(252)
            rolling_sharpe = rolling_returns / rolling_vol
            
            plt.figure(figsize=(12, 6))
            rolling_returns.plot()
            plt.title('Retorno Anualizado Móvil (252 días)')
            plt.xlabel('Fecha')
            plt.ylabel('Retorno Anualizado')
            plt.grid(True)
            plt.savefig(f'{save_path}rolling_returns.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            # 12. MEJORA: Sharpe ratio móvil
            plt.figure(figsize=(12, 6))
            rolling_sharpe.plot()
            plt.axhline(y=0, color='r', linestyle='--')
            plt.title('Sharpe Ratio Móvil (252 días)')
            plt.xlabel('Fecha')
            plt.ylabel('Sharpe Ratio')
            plt.grid(True)
            plt.savefig(f'{save_path}rolling_sharpe.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Gráficos guardados en {save_path}")
            
        except Exception as e:
            logging.error(f"Error en plot_results: {str(e)}", exc_info=True)
    
    def run_walk_forward_analysis(self, train_size=0.6, step_size=63, train_lookback=252):
        """
        MEJORA: Ejecuta análisis walk-forward más robusto para evaluar la estrategia.
        
        Parámetros:
        -----------
        train_size : float
            Proporción de datos a usar para entrenamiento en cada ventana
        step_size : int
            Tamaño del paso para avanzar la ventana de prueba (en días)
        train_lookback : int
            Cantidad de días máximos a utilizar en cada ventana de entrenamiento
            
        Retorna:
        --------
        wfa_results : DataFrame
            Resultados del análisis walk-forward
        """
        try:
            # Asegurar que tenemos suficientes datos
            if len(self.returns) < self.lookback_window + 2 * step_size:
                raise ValueError("No hay suficientes datos para análisis walk-forward")
            
            # Inicializar resultados
            wfa_results = []
            dates = self.returns.index
            
            # Definir ventanas
            start_idx = self.lookback_window
            while start_idx + step_size < len(dates):
                # Limitar la ventana de entrenamiento
                train_end_idx = start_idx + int((len(dates) - start_idx) * train_size)
                train_start_idx = max(0, train_end_idx - train_lookback)
                test_end_idx = min(train_end_idx + step_size, len(dates))
                
                train_start_date = dates[train_start_idx]
                train_end_date = dates[train_end_idx - 1]
                test_start_date = dates[train_end_idx]
                test_end_date = dates[test_end_idx - 1]
                
                print(f"\nVentana WFA: {test_start_date.strftime('%Y-%m-%d')} a {test_end_date.strftime('%Y-%m-%d')}")
                
                # MEJORA: Guardar configuración actual y usar parámetros optimizados para cada ventana
                original_n_regimes = self.n_regimes
                original_lookback = self.lookback_window
                original_regime_method = self.regime_detection_method
                
                # Ajustar parámetros según la ventana
                # Más regímenes para períodos volátiles
                train_window_vol = self.returns.loc[train_start_date:train_end_date].std().mean() * np.sqrt(252)
                if train_window_vol > 0.25:  # Alta volatilidad
                    self.n_regimes = 6
                    self.lookback_window = 126  # Ventana más corta
                    self.regime_detection_method = 'bgmm'
                elif train_window_vol > 0.15:  # Volatilidad media
                    self.n_regimes = 5
                    self.lookback_window = 189
                    self.regime_detection_method = 'bgmm'
                else:  # Baja volatilidad
                    self.n_regimes = 4
                    self.lookback_window = 252
                    self.regime_detection_method = 'gmm'
                
                # Ejecutar backtest en datos de entrenamiento
                train_performance = self.backtest(
                    start_date=train_start_date,
                    end_date=train_end_date
                )
                
                # Guardar pesos óptimos del último rebalanceo
                last_weights = self.weights_history.iloc[-1]
                
                # Considerar costos de transacción al aplicar pesos
                initial_turnover = np.sum(np.abs(last_weights))
                initial_cost = initial_turnover * self.transaction_cost
                
                # Inicializar tracking de posiciones cortas
                short_positions = last_weights[last_weights < 0]
                daily_borrow_cost = self.borrow_cost / 252
                
                # Ejecutar backtest en datos de prueba con pesos fijos
                test_returns = self.returns.loc[test_start_date:test_end_date]
                test_portfolio_values = [1.0 - initial_cost]
                
                for date, returns in test_returns.iterrows():
                    # Calcular costo de posiciones cortas
                    if not short_positions.empty:
                        short_cost = (short_positions.abs() * daily_borrow_cost).sum()
                    else:
                        short_cost = 0
                    
                    # Calcular retorno del portafolio con costos
                    portfolio_return = (last_weights * returns).sum() - short_cost
                    test_portfolio_values.append(test_portfolio_values[-1] * (1 + portfolio_return))
                
                # Calcular métricas para esta ventana
                test_returns_series = pd.Series(
                    [test_portfolio_values[i+1]/test_portfolio_values[i] - 1 for i in range(len(test_portfolio_values)-1)],
                    index=test_returns.index
                )
                
                test_performance = pd.DataFrame({
                    'Returns': test_returns_series,
                    'Cumulative_Returns': (1 + test_returns_series).cumprod()
                })
                
                test_performance['Drawdown'] = 1 - test_performance['Cumulative_Returns'] / test_performance['Cumulative_Returns'].cummax()
                
                # Calcular métricas
                total_return = test_performance['Cumulative_Returns'].iloc[-1] - 1
                ann_factor = 252
                ann_return = (1 + total_return) ** (ann_factor / len(test_returns_series)) - 1
                ann_volatility = test_returns_series.std() * np.sqrt(ann_factor)
                sharpe_ratio = ann_return / ann_volatility if ann_volatility > 0 else 0
                max_drawdown = test_performance['Drawdown'].max()
                
                # Guardar resultados
                wfa_results.append({
                    'Test_Start_Date': test_start_date,
                    'Test_End_Date': test_end_date,
                    'Total_Return': total_return,
                    'Annualized_Return': ann_return,
                    'Annualized_Volatility': ann_volatility,
                    'Sharpe_Ratio': sharpe_ratio,
                    'Max_Drawdown': max_drawdown,
                    'Initial_Cost': initial_cost,
                    'Short_Exposure': short_positions.abs().sum() if not short_positions.empty else 0,
                    'Train_Window_Vol': train_window_vol,
                    'N_Regimes_Used': self.n_regimes,
                    'Lookback_Used': self.lookback_window,
                    'Method_Used': self.regime_detection_method
                })
                
                # Restaurar parámetros originales
                self.n_regimes = original_n_regimes
                self.lookback_window = original_lookback
                self.regime_detection_method = original_regime_method
                
                # Avanzar ventana
                start_idx = train_end_idx
            
            # Convertir resultados a DataFrame
            wfa_df = pd.DataFrame(wfa_results)
            
            # Guardar resultados
            wfa_df.to_csv('./artifacts/results/data/walk_forward_analysis.csv', index=False)
            
            # Calcular métricas agregadas
            wfa_metrics = {
                'Mean_Sharpe': wfa_df['Sharpe_Ratio'].mean(),
                'Median_Sharpe': wfa_df['Sharpe_Ratio'].median(),
                'Min_Sharpe': wfa_df['Sharpe_Ratio'].min(),
                'Max_Sharpe': wfa_df['Sharpe_Ratio'].max(),
                'Mean_Return': wfa_df['Annualized_Return'].mean(),
                'Mean_Volatility': wfa_df['Annualized_Volatility'].mean(),
                'Mean_Drawdown': wfa_df['Max_Drawdown'].mean(),
                'Consistency': (wfa_df['Sharpe_Ratio'] > 0).mean(),
                'Mean_Initial_Cost': wfa_df['Initial_Cost'].mean(),
                'Mean_Short_Exposure': wfa_df['Short_Exposure'].mean()
            }
            
            # Guardar métricas agregadas
            pd.Series(wfa_metrics).to_csv('./artifacts/results/data/walk_forward_metrics.csv')
            
            # Visualizar resultados
            plt.figure(figsize=(12, 8))
            plt.subplot(2, 1, 1)
            
            # Colorear barras por sharpe ratio (rojo si negativo, verde si positivo)
            colors = ['green' if sr > 0 else 'red' for sr in wfa_df['Sharpe_Ratio']]
            plt.bar(range(len(wfa_df)), wfa_df['Sharpe_Ratio'], color=colors)
            
            plt.axhline(y=0, color='r', linestyle='-')
            plt.title('Sharpe Ratio por Ventana de Prueba')
            plt.xticks(range(len(wfa_df)), [d.strftime('%Y-%m') for d in wfa_df['Test_Start_Date']], rotation=45)
            plt.grid(True)
            
            plt.subplot(2, 1, 2)
            colors = ['green' if r > 0 else 'red' for r in wfa_df['Total_Return']]
            plt.bar(range(len(wfa_df)), wfa_df['Total_Return'], color=colors)
            plt.axhline(y=0, color='r', linestyle='-')
            plt.title('Retorno Total por Ventana de Prueba')
            plt.xticks(range(len(wfa_df)), [d.strftime('%Y-%m') for d in wfa_df['Test_Start_Date']], rotation=45)
            plt.grid(True)
            
            plt.tight_layout()
            plt.savefig('./artifacts/results/figures/walk_forward_results.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            return wfa_df
            
        except Exception as e:
            logging.error(f"Error en run_walk_forward_analysis: {str(e)}", exc_info=True)
            return pd.DataFrame()

# Ejecutar la estrategia
if __name__ == "__main__":
    try:
        # Inicializar estrategia con parámetros mejorados
        strategy = AdaptiveMultifactorStrategy(
            start_date='2015-01-01',
            end_date='2025-01-01',
            lookback_window=189,                # Más corto que 252 para adaptación más rápida
            regime_window=63,                   # Más corto para detectar cambios de régimen
            n_regimes=5,                        # Más regímenes para mejor granularidad
            rebalance_freq=5,                   # 5 dias de trading
            vol_target=0.10,                    # 10% anualizado
            max_leverage=1.5,                   # Límite de apalancamiento
            transaction_cost=0.0005,            # 5 puntos básicos
            market_impact=0.1,                  # Como factor de volatilidad diaria
            borrow_cost=0.0002,                 # 20 puntos básicos anualizados
            execution_delay=1,                  # 1 día de retraso
            use_point_in_time=False,            # No usar point-in-time data
            max_drawdown_limit=0.20,            # Limitar drawdown al 20%
            dynamic_vol_scaling=True,           # Escalar volatilidad dinámicamente
            risk_targeting=True,                # Usar targeting de riesgo basado en régimen
            regime_detection_method='bgmm'      # Usar Bayesian GMM para detección de regímenes
        )
        
        # Ejecutar backtest
        performance = strategy.backtest()
        
        # Calcular métricas
        metrics = strategy.calculate_metrics()
        print("\nMétricas de Rendimiento:")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")
        
        # Generar visualizaciones
        strategy.plot_results()
        
        # Ejecutar análisis walk-forward
        wfa_results = strategy.run_walk_forward_analysis(train_size=0.6, step_size=63, train_lookback=252)
        
        print("\nAnálisis completado. Todos los resultados guardados en ./artifacts/results/")
        
    except Exception as e:
        logging.error(f"Error en la ejecución principal: {str(e)}", exc_info=True)
        print(f"Error: {str(e)}. Ver ./artifacts/errors.txt para más detalles.")

[*********************100%***********************]  503 of 503 completed


Datos cargados exitosamente. 472 símbolos, 2315 días de trading.
En promedio, 99.9% de los activos son negociables.
En promedio, 99.9% de los activos son susceptibles de posiciones cortas.


  0%|▎                                                                                        | 6/2126 [12:45<72:25:21, 122.98s/it]

Rebalanceo por cambio de régimen: 3 -> 0


  1%|▍                                                                                        | 11/2126 [14:44<39:50:30, 67.82s/it]

...

 63%|█████████████████████████████████████████████████████▊                               | 1346/2126 [19:41:06<5:06:38, 23.59s/it]

AVISO: Apalancamiento corregido de 4.39 a 1.50


 64%|██████████████████████████████████████████████████████                               | 1351/2126 [19:42:29<4:37:40, 21.50s/it]

AVISO: Apalancamiento corregido de 4.50 a 1.50
Rebalanceo por cambio de régimen: 0 -> 4


 64%|██████████████████████████████████████████████████████▏                              | 1356/2126 [19:43:50<4:15:50, 19.94s/it]

In [ ]:
print("DONE")